# Chapter 13

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model


In [ ]:
(img_train, _), (img_test, _) = mnist.load_data()

img_train = img_train.astype("float32") / 255.0
img_train = np.reshape(img_train, (len(img_train), 28, 28, 1))

img_test = img_ test.astype("float32") / 255.0
img_ test = np.reshape(img_ test, (len(img_ test), 28, 28, 1))


In [ ]:
noise_factor = 0.4
noisy_train = img_train + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size= img_train.shape)
noisy_train = np.clip(noisy_train, 0.0, 1.0)

noisy_test = img_test + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size= img_test.shape)
noisy_test = np.clip(noisy_test, 0.0, 1.0)


In [ ]:
model = Sequential()

model.add(InputLayer(input_shape=(28, 28, 1)))

# Encoder
model.add(Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(MaxPooling2D((2, 2), padding="same"))
model.add(Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(MaxPooling2D((2, 2), padding="same"))

# Decoder
model.add(Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same"))
model.add(Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same"))
model.add(Conv2D(1, (3, 3), activation="sigmoid", padding="same"))

# Compile the auto-encoder
model.compile(optimizer="adam", loss="binary_crossentropy")

model.summary()


In [ ]:
model.fit(img_train, img_train, epochs=2, batch_size=128, validation_data=( img_test, img_test))
predictions = model.predict(img_test)

In [ ]:
model.fit(noisy_train, img_train, epochs=2, batch_size=128, validation_data=(noisy_test, img_test))
predictions = model.predict(noisy_test)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds


from keras.layers import Input, Dense, Conv2D, BatchNormalization, UpSampling2D
from keras.layers import Dropout, MaxPooling2D, Rescaling, ReLU, LeakyReLU
from keras.layers import Add, Flatten, Activation, Reshape, Conv2DTranspose
from keras.models import Model
#import keras_tuner as kt

**For the discriminator** as recommended in the GAN literature, we are using strided convolutions and LeakyReLU activations. The use of dropout layers will allow to control the overfit. The last layer of the network has one unit output and a sigmoid activation given that the discriminator’s task is to perform a binary classification and recognize whether the image is real (label of 1) or generated (label of 0).

Description of the layers of discriminator:
We are using 4 strided convolutions and LeakyReLU activations:
  - Kernel size: 5
  - Stride: 2
  - Padding: same
  - Activation: LeakyReLU(alpha=0.2)

The number of filters is respectively 64, 128, 256, 512.
After each convolution we add a dropout layer to avoid overfitting.

Then build the binary classification head that should output a probability of
a real image from 0 to 1

In [ ]:
discriminator = tf.keras.Sequential()
dropout_rate = 0.5

input_shape = (28, 28, 1)

# Write your code below
discriminator.add(Conv2D(64, 5, strides=2, input_shape=input_shape, padding='same', activation=LeakyReLU(alpha=0.2)))
discriminator.add(Dropout(dropout_rate))
discriminator.add(Conv2D(128, 5, strides=2, padding='same', activation=LeakyReLU(alpha=0.2)))
discriminator.add(Dropout(dropout_rate))
discriminator.add(Conv2D(256, 5, strides=2, padding='same', activation=LeakyReLU(alpha=0.2)))
discriminator.add(Dropout(dropout_rate))
discriminator.add(Conv2D(512, 5, strides=1, padding='same', activation=LeakyReLU(alpha=0.2)))
discriminator.add(Dropout(dropout_rate))

discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 14, 14, 64)        1664      
                                                                 
 dropout_5 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 dropout_6 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 4, 4, 256)         819456    
                                                                 
 dropout_7 (Dropout)         (None, 4, 4, 256)         0         
                                                                 
 conv2d_7 (Conv2D)           (None, 4, 4, 512)        

**For the generator** We are using up-sampling layers together with learned Conv2DTranspose layers. The usage of BatchNormalization layers is important to stabilize the training process. We can notice the final layer that is a sigmoid layer that receives the output of the last convolution layer. This is so because the output of the generator will be a grayscale image.

In [ ]:
generator = tf.keras.Sequential()
dropout = 0.5
depth = 64+64+64+64
dim = 7
# In: 100
# Out: dim x dim x depth

# MLP network
generator.add(Dense(7*7*256, input_dim=100))
# Use batchnormalization to stabilize the learning
generator.add(BatchNormalization(momentum=0.9))
generator.add(Activation('relu'))

# Upscaling network
generator.add(Reshape((7, 7, 256)))
generator.add(Dropout(dropout))
# In: dim x dim x depth
# Out: 2*dim x 2*dim x depth/2
generator.add(UpSampling2D())
generator.add(Conv2DTranspose(128, 5, padding='same'))
generator.add(BatchNormalization(momentum=0.9))
generator.add(Activation('relu'))
generator.add(UpSampling2D())
generator.add(Conv2DTranspose(64, 5, padding='same'))
generator.add(BatchNormalization(momentum=0.9))
generator.add(Activation('relu'))
generator.add(Conv2DTranspose(32, 5, padding='same'))
generator.add(BatchNormalization(momentum=0.9))
generator.add(Activation('relu'))

# Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
generator.add(Conv2DTranspose(1, 5, padding='same'))
generator.add(Activation('sigmoid'))
generator.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 12544)             1266944   
                                                                 
 batch_normalization_4 (Bat  (None, 12544)             50176     
 chNormalization)                                                
                                                                 
 activation_5 (Activation)   (None, 12544)             0         
                                                                 
 reshape_1 (Reshape)         (None, 7, 7, 256)         0         
                                                                 
 dropout_9 (Dropout)         (None, 7, 7, 256)         0         
                                                                 
 up_sampling2d_2 (UpSamplin  (None, 14, 14, 256)       0         
 g2D)                                                 

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0008, clipvalue=1.0, weight_decay=6e-8)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer,\
metrics=['accuracy'])

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0004, clipvalue=1.0, weight_decay=3e-8)

adverse = tf.keras.Sequential()
adverse.add(generator)
adverse.add(discriminator)

adverse.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
(img_train, label_train), (img_test, label_test) = tf.keras.datasets.fashion_mnist.load_data()

img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

img_train = np.expand_dims(img_train, axis=3)


4422102/4422102 [==============================] - 0s 0us/step


We need now to define the learning algorithm. The process that we will follow is to train the discriminator alone in a first stage. The discriminator will learn to recognize real images from generated images. Below is the learning algorithm

In [ ]:
for _ in range(200):
  batch_size=64

  images_train = img_train[np.random.randint(0,img_train.shape[0], size=batch_size), :, :, :]
  noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
  images_fake = generator.predict(noise)

  x = np.concatenate((images_train, images_fake))
  y = np.ones([2*batch_size, 1])
  y[batch_size:, :] = 0

  d_loss = discriminator.train_on_batch(x, y)
  print(d_loss)


2/2 [==============================] - 0s 5ms/step
[0.6935915350914001, 0.4609375]
2/2 [==============================] - 0s 5ms/step
[0.6660510301589966, 0.5703125]
2/2 [==============================] - 0s 4ms/step
[0.588917076587677, 0.7109375]
2/2 [==============================] - 0s 5ms/step
[1.2344748973846436, 0.5]
2/2 [==============================] - 0s 4ms/step
[1.8340023756027222, 0.5]
2/2 [==============================] - 0s 4ms/step
[0.740145742893219, 0.3984375]
2/2 [==============================] - 0s 4ms/step
[2.6933960914611816, 0.5]
2/2 [==============================] - 0s 4ms/step
[0.6015135049819946, 0.953125]
2/2 [==============================] - 0s 4ms/step
[0.5449613928794861, 0.890625]
2/2 [==============================] - 0s 4ms/step
[0.4492356479167938, 0.90625]
2/2 [==============================] - 0s 4ms/step
[0.3358818292617798, 0.9453125]
2/2 [==============================] - 0s 4ms/step
[0.2831079065799713, 0.8828125]
2/2 [=======================

In a second step, we will jointly train our discriminator and generator. When training the discriminator, the setup will be the same as above. When training the generator, we want to reward it when it manages to fool the discriminator. As a result, the labels will be reversed. So, the label of a generated image will be 1. We will also need to chain the generator and the discriminator so that the output of the former is connected to the input of the latter:

In [ ]:
for _ in range(1000):
  batch_size=64
  real_images = img_train[np.random.randint(0,img_train.shape[0], size=batch_size), :, :, :]

  noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
  generated_images = generator.predict(noise)

  x = np.concatenate((real_images, generated_images))
  y = np.ones([2*batch_size, 1])
  y[batch_size:, :] = 0

  d_loss = discriminator.train_on_batch(x, y)

  y = np.ones([batch_size, 1])
  noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])

  a_loss = adverse.train_on_batch(noise, y)
  print(a_loss)


2/2 [==============================] - 0s 5ms/step
[0.48723068833351135, 0.765625]
2/2 [==============================] - 0s 4ms/step
[1.916372299194336, 0.109375]
2/2 [==============================] - 0s 6ms/step
[0.727758526802063, 0.59375]
2/2 [==============================] - 0s 4ms/step
[1.6864690780639648, 0.203125]
2/2 [==============================] - 0s 7ms/step
[1.1469907760620117, 0.46875]
2/2 [==============================] - 0s 5ms/step
[1.3332595825195312, 0.25]
2/2 [==============================] - 0s 5ms/step
[1.492377758026123, 0.234375]
2/2 [==============================] - 0s 9ms/step
[0.8874094486236572, 0.5]
2/2 [==============================] - 0s 7ms/step
[2.0164761543273926, 0.171875]
2/2 [==============================] - 0s 5ms/step
[1.018782377243042, 0.5]
2/2 [==============================] - 0s 5ms/step
[2.207098960876465, 0.109375]
2/2 [==============================] - 0s 5ms/step
[2.2906064987182617, 0.203125]
2/2 [==============================]

In [ ]:
noise = np.random.uniform(-1.0, 1.0, size=[9, 100])
images_generated = generator.predict(noise)
images_generated = images_generated *255
images_generated = images_generated.astype(np.uint8)

1/1 [==============================] - 0s 20ms/step


In [ ]:
import cv2
fig, ax = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(10,10))
for i in range(9):
  img = images_generated[i]
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  ax[i%3][i//3].imshow(img_rgb)
  i += 1
plt.show()